In [1]:
import numpy as np  # Librería para aplicar álgebra lineal
import pandas as pd # Para manejar los datos (datasets)

import matplotlib.pyplot as plt # Para las visualizaciones
import seaborn as sns # Visualizaciones más fancy

'''from sklearn import preprocessing # Para el procesador de los datos
from sklearn.preprocessing import Imputer # Para adoptar una estrategia para los missing values
from sklearn.preprocessing import LabelEncoder as Codificar # Para codificar variables categóricas
from sklearn.preprocessing import OneHotEncoder # Para pasar el LaberEncoder vector a OneHot matriz
from sklearn.preprocessing import MinMaxScaler # Para realizar el escalado en escala (0-1)
from sklearn.model_selection import train_test_split as Separar # Para dividir en los 2 conjuntos
#from statsmodels.tools.eval_measures import rmse # Para calcular el error
from sklearn.metrics import confusion_matrix as CM # Para construir la matriz de confusión
from matplotlib.colors import ListedColormap as Colors # Para pintar las regiones en Clasificación'''

'from sklearn import preprocessing # Para el procesador de los datos\nfrom sklearn.preprocessing import Imputer # Para adoptar una estrategia para los missing values\nfrom sklearn.preprocessing import LabelEncoder as Codificar # Para codificar variables categóricas\nfrom sklearn.preprocessing import OneHotEncoder # Para pasar el LaberEncoder vector a OneHot matriz\nfrom sklearn.preprocessing import MinMaxScaler # Para realizar el escalado en escala (0-1)\nfrom sklearn.model_selection import train_test_split as Separar # Para dividir en los 2 conjuntos\n#from statsmodels.tools.eval_measures import rmse # Para calcular el error\nfrom sklearn.metrics import confusion_matrix as CM # Para construir la matriz de confusión\nfrom matplotlib.colors import ListedColormap as Colors # Para pintar las regiones en Clasificación'

In [2]:
#Datos de entrada

#Se especifican los tipo de datos para mejorar la performance
eventos = pd.read_csv('../TP2018/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv', 
                      dtype={"timestamp": object,#intenté hacerla datetime y no anduvo 
                             "event": 'category',
                             "person":object,
                             "url":object,
                             "sku":object,
                             "model":object,
                             "condition":'category',
                             "storage":object,
                             "color":'category',
                             "skus":object,
                             "search_term":object,
                             "staticpage":object,
                             "campaign_source":object,
                             "search_engine":object,
                             "channel":object,
                             "new_vs_returning":'category',
                             "city":object,
                             "region":object,
                             "country":object,
                             "device_type":object,
                             "screen_resolution":object,
                             "operating_system_version":object,
                             "browser_version":object})

labels = pd.read_csv('../TP2018/fiuba-trocafone-tp2-final-set/labels_training_set.csv',
                        dtype={"person":object, "label":bool})

clientesAEvaluar = pd.read_csv('../TP2018/fiuba-trocafone-tp2-final-set/trocafone_kaggle_test.csv')

In [3]:
#Revisando el formato de los eventos
eventos.head(3)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#Confirmando que coinnciden la cantidad de datos a clasificar
print('Cantidad de clientes que no sabemos:', len(eventos['person'].unique())- len(labels))
print('Cantidad de clientes a clasificar:',len(clientesAEvaluar))

Cantidad de clientes que no sabemos: 19415
Cantidad de clientes a clasificar: 19415


In [9]:
#Analizando los datos de entrenamiento
print('Cantidades de conversiones:')
print(labels['label'].value_counts())
print('Proporción de conversiones:')
print(labels['label'].value_counts(normalize=True))

Cantidades de conversiones:
False    18434
True       980
Name: label, dtype: int64
Proporción de conversiones:
False    0.949521
True     0.050479
Name: label, dtype: float64


In [10]:
#Agregamos algunos features a los existentes
eventos['timestamp'] =  pd.to_datetime(eventos['timestamp'])
eventos[['marca','modelo']] = eventos['model'].dropna().str.split(' ',n=1,expand=True)
eventos['weekday'] = eventos['timestamp'].dt.day_name()
eventos['hour'] = eventos['timestamp'].dt.hour
eventos['month'] = eventos['timestamp'].dt.month
eventos['day'] = eventos['timestamp'].dt.day

### Vamos creando features: 

In [21]:
#Features en base al total de cada evento
totalEvento = eventos.groupby('person')['event'].value_counts().unstack().reset_index()
totalEvento.rename(lambda x: x if x == 'person' else 'total ' + x, axis = 1, inplace=True)

#Features en base a la frecuencia de cada evento sobre el total (frecuencia relativa)
frecEvento = eventos.groupby('person')['event'].value_counts(normalize=True).unstack().reset_index()
frecEvento.rename(lambda x: x if x == 'person' else 'frec ' + x, axis = 1, inplace=True)

#features en base a la frecuencia relativa de la marca de los dispositivos consultados
frecMarca = eventos.groupby('person')['marca'].value_counts(normalize=True).unstack().reset_index()
frecMarca.rename(lambda x: x if x == 'person' else 'frec ' + x, axis = 1, inplace=True)

#features en base a la cantidad (frecuencia total) de eventos por mes
totalMes = eventos.groupby('person')['month'].value_counts().unstack().reset_index()
totalMes.rename({1:'total_enero',2:'total_febrero',3:'total_marzo',4:'total_abril',5:'total_mayo'}, 
                inplace=True, axis=1)

#features en base a la frecuencia relativa de los dispositivos desde los que se accede
frecDispositivo = eventos.groupby('person')['device_type'].value_counts(normalize=True).unstack().reset_index()
frecDispositivo.rename(lambda x: x if x == 'person' else 'frec ' + x, axis=1, inplace=True)

#features en base a la frecuencia relativa de los colores de los teléfonos vistos
frecColor = eventos.groupby('person')['color'].value_counts(normalize=True).unstack().reset_index()
frecColor.rename(lambda x: x if x == 'person' else 'frec color ' + x, axis=1, inplace=True)

#features en base a la frecuencia relativa de los estados de los teléfonos consultados
frecCondicion = eventos.groupby('person')['condition'].value_counts(normalize=True).unstack().reset_index()
frecCondicion.rename(lambda x: x if x == 'person' else 'frec ' + x, axis=1, inplace=True)

#features en base al la frecuencia relativa del día de semana en que se consultó
frecDiaSemana = eventos.groupby('person')['weekday'].value_counts(normalize=True).unstack().reset_index().fillna(0)
frecDiaSemana.rename(lambda x: x if x == 'person' else 'frec ' + x, axis=1, inplace=True)

#features en base al día (a revisar, capaz convenga hacer algo en base al més y al día, o separar en quincenas)
frecNroDia = eventos.groupby('person')['day'].value_counts(normalize=True).unstack().reset_index()
frecNroDia.rename(lambda x: x if str(x) == 'person' else 'frec dia ' + str(x), axis=1, inplace=True)

#Creamos los features del almacenamiento interno de los dispositivos consultados
frecAlmacenamiento = eventos.groupby('person')['storage'].value_counts(normalize=True).unstack().reset_index()
frecAlmacenamiento.rename(lambda x: x if x == 'person' else 'frec ' + x, axis=1, inplace=True)


In [23]:
#Empezamos a hacer el merge
#No hay que olvidarse de hacer un outer left joins, si no se perderán filas
#Empezamos con features con todas las filas (todos los clientes tienen eventos)
features = frecEvento.merge(totalEvento, how='left', on='person')
features = features.merge(frecMarca, how='left', on='person')
features = features.merge(totalMes, how='left', on='person')
features = features.merge(frecDispositivo, how='left', on='person')
features = features.merge(frecColor, how='left', on='person')
features = features.merge(frecCondicion, how='left', on='person')
features = features.merge(frecDiaSemana, how='left', on='person')
features = features.merge(frecNroDia, how='left', on='person')
features = features.merge(frecAlmacenamiento, how='left', on='person')

features.head(2)






,person,frec ad campaign hit,frec brand listing,frec checkout,frec conversion,frec generic listing,frec lead,frec search engine hit,frec searched products,frec staticpage,...,frec dia 30,frec dia 31,frec 128GB,frec 16GB,frec 256GB,frec 32GB,frec 4GB,frec 512MB,frec 64GB,frec 8GB
0,0008ed71,NaN,NaN,0.500000,NaN,0.166667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.666667,NaN,NaN,0.333333,NaN
1,00091926,0.033482,0.055804,0.004464,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.035714,0.128342,0.278075,0.026738,0.352941,NaN,NaN,0.213904,NaN


#### Seguir con feature engeneering, agregando features. Algunas ideas: 
*    Modelo más consultado
*    dividir el mes en 3 o 4 y ver cuantos eventos se generaron en determinadas epocas del mes
*    Modelo mas consultado
*    navegador más usado (capaz ver los 3 o 5 más frecuentes y una columna otros)
*    etc

## Machine learning
### Primer algoritmo usado: Random Forest

In [25]:
#Reemplazo NaNs por ceros
features = features.fillna(0)

#Cargamos X e y para trabajar con Random Forest (y otros algoritmos luego).
X = pd.merge(features, labels, on='person')
X.drop('person', axis=1, inplace=True)
X = X.fillna(0)
y = X.pop('label')


In [26]:
'''Tomamos "prestada" una función para presentar prolijamente los n features más importantes.
La función fue tomada casi literal de:
    https://github.com/Featuretools/predict-next-purchase/blob/master/utils.py
'''
def feature_importances(model, features, n=10):
    importances = model.feature_importances_
    zipped = sorted(zip(features, importances), key=lambda x: -x[1])
    for i, f in enumerate(zipped[:n]):
        print("%d: Feature: %s, %.3f" % (i+1, f[0], f[1]))

    return [f[0] for f in zipped[:n]]

In [27]:
#Cargamos las bibliotecas para usar y validar Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [28]:
#Creamos el clasificador
'''Nota: Tomamos una configuración de random forest encontrada en:
    https://www.datacamp.com/community/tutorials/random-forests-classifier-python'''

clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
#Otra version:
#clf = RandomForestClassifier(n_estimators=400, n_jobs=-1)

In [29]:
#Probamos qué tan bien viene clasificando nuestro modelo

scores = cross_val_score(estimator=clf,X=X, y=y, cv=3,
                         scoring="roc_auc", verbose=True)

"AUC %.2f +/- %.2f" % (scores.mean(), scores.std())

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   44.7s finished


'AUC 0.85 +/- 0.01'

In [30]:
#Vemos cuales fueron los features más relevantes, aprovechando las bondades de Random Forests
clf.fit(X,y)
top_features = feature_importances(clf, X.columns, n=20)

1: Feature: frec checkout, 0.039
2: Feature: total checkout, 0.033
3: Feature: total_mayo, 0.029
4: Feature: frec viewed product, 0.022
5: Feature: frec visited site, 0.021
6: Feature: total viewed product, 0.019
7: Feature: frec ad campaign hit, 0.019
8: Feature: frec generic listing, 0.019
9: Feature: frec search engine hit, 0.019
10: Feature: frec brand listing, 0.017
11: Feature: frec Bom, 0.017
12: Feature: frec color Dourado, 0.016
13: Feature: frec Muito Bom, 0.016
14: Feature: frec 16GB, 0.016
15: Feature: frec color Preto, 0.016
16: Feature: total brand listing, 0.015
17: Feature: frec 32GB, 0.015
18: Feature: frec Excelente, 0.014
19: Feature: total generic listing, 0.014
20: Feature: total ad campaign hit, 0.014


In [31]:
#vemos cómo quedaría una pequeña muestra de la matriz de features con los más relevantes
X.loc[:,top_features].head(2)

,frec checkout,total checkout,total_mayo,frec viewed product,frec visited site,total viewed product,frec ad campaign hit,frec generic listing,frec search engine hit,frec brand listing,frec Bom,frec color Dourado,frec Muito Bom,frec 16GB,frec color Preto,total brand listing,frec 32GB,frec Excelente,total generic listing,total ad campaign hit
0,0.500000,3.0,6.0,0.000000,0.333333,0.0,0.000000,0.166667,0.000000,0.0,0.666667,0.333333,0.333333,0.0,0.333333,0.0,0.666667,0.0,1.0,0.0
1,0.058824,1.0,17.0,0.176471,0.058824,3.0,0.058824,0.058824,0.058824,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0,1.0


In [32]:
#Pruebo de nuevo los random forest con los resultados obtenidos de mejores clasificadores
X_reducido = X.loc[:,top_features]
scores = cross_val_score(estimator=clf,X=X_reducido, y=y, cv=3,
                         scoring="roc_auc", verbose=True)

"AUC %.2f +/- %.2f" % (scores.mean(), scores.std())

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   24.4s finished


'AUC 0.81 +/- 0.01'

In [35]:
#Veamos cuantos predice que convierten en los 15 primeros días de junio
clf.fit(X,y)
prediccionTest = clf.predict(X=X)
print('total del set de entrenamiento que convirtieron:', labels['label'].sum())
print('total del set de entrenamiento que predice que convertirán:',pd.Series(prediccionTest).sum())

total del set de entrenamiento que convirtieron: 980
total del set de entrenamiento que predice que convertirán: 979


## Vamos a probar otro clasificador: XGBoost

In [39]:
#Primero importamos las bibliotecas que necesitaremos
import xgboost as xgb

#Para preparar sets de entrenamiento y prueba
from sklearn.model_selection import train_test_split  

#Nos traemos bibliotecas para medir la performance
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


In [37]:
#Hacemos una división de set de test y entrenamiento, y le ponemos una semilla arbitraria por si queremos replicarlo
seed = 1234
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [44]:
#Probamos el modelo más sencillo de XGBoost, con los parámetros por defecto
modeloBasico = xgb.XGBClassifier()
train_modeloBasico = modeloBasico.fit(X_train, y_train)


mejor_n_limit = 0
mejor_precision = 0
for i in range(200):
    prediccion = train_modeloBasico.predict_proba(X_test, ntree_limit=i)
    precision = accuracy_score(y_test, prediccion[:,[1]] > 0.5) * 100
    if(precision > mejor_precision):
        mejor_precision = precision
        mejor_n_limit = i
    
print("Precisión modeloBasico con ntree_limit=",mejor_n_limit,": %.2f" % mejor_precision)

scores = cross_val_score(estimator=train_modeloBasico,X=X, y=y, cv=3,
                         scoring="roc_auc", verbose=True)
print("AUC %.2f +/- %.2f" % (scores.mean(), scores.std()))


Precisión modeloBasico con ntree_limit= 81 : 95.93
AUC 0.85 +/- 0.00


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   25.6s finished


## Preparamos los datos que necesitamos clasificar

In [46]:
datosTest = clientesAEvaluar.merge(features, on='person',how='inner')
print('cantidad de datos:',len(datosTest))
datosTest.head(2)

cantidad de datos: 19415


,person,frec ad campaign hit,frec brand listing,frec checkout,frec conversion,frec generic listing,frec lead,frec search engine hit,frec searched products,frec staticpage,...,frec dia 30,frec dia 31,frec 128GB,frec 16GB,frec 256GB,frec 32GB,frec 4GB,frec 512MB,frec 64GB,frec 8GB
0,4886f805,0.000000,0.000000,0.111111,0.0,0.111111,0.000000,0.111111,0.111111,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.0
1,0297fc1e,0.051146,0.007055,0.012346,0.0,0.037037,0.001764,0.000000,0.010582,0.0,...,0.024691,0.007055,0.121655,0.311436,0.014599,0.082725,0.0,0.0,0.469586,0.0


In [48]:
#Verifico que no se hayan "perdido" filas en el camino (un merge mal definido capaz?)
print('cantidad de personas en total:',len(eventos['person'].unique()))
print('cantidad de personas para training:',len(labels))
print('cantidad de personas para evaluar:',len(clientesAEvaluar))
print('cantidad de personas en clientes agrupados:',len(features))

cantidad de personas en total: 38829
cantidad de personas para training: 19414
cantidad de personas para evaluar: 19415
cantidad de personas en clientes agrupados: 38829


In [49]:
#Haciendo las predicciones con el modelo de Random Forest
clf.fit(X,y)
prediccionRandomForest = clf.predict_proba(X=datosTest.drop('person', axis=1).fillna(0))

In [51]:
for i in range(1,6):
    print('%d\tclientes con p > %.2f de convertir' % ((prediccionRandomForest[:,[1]] > i/10).sum(),i/10))

3689	clientes con p > 0.10 de convertir
1183	clientes con p > 0.20 de convertir
224	clientes con p > 0.30 de convertir
29	clientes con p > 0.40 de convertir
3	clientes con p > 0.50 de convertir


In [52]:
#Haciendo predicciones con el modelo básico de XGBoost
prediccionXGBoostBasico = train_modeloBasico.predict_proba(data=datosTest.drop('person', axis=1).fillna(0),
                                                           ntree_limit=mejor_n_limit) 
#Habrá que ver si no "overfittea" al ponerle el mejor n_limit

In [53]:
for i in range(1,6):
    print('%d\tclientes con p > %.2f de convertir' % ((prediccionXGBoostBasico[:,[1]] > i/10).sum(),i/10))

3378	clientes con p > 0.10 de convertir
1089	clientes con p > 0.20 de convertir
266	clientes con p > 0.30 de convertir
115	clientes con p > 0.40 de convertir
76	clientes con p > 0.50 de convertir


In [54]:
#Nos quedamos con el modelo de XGBoost
clientesAEvaluar['label'] = prediccionXGBoostBasico[:,[1]]
clientesAEvaluar.head()

,person,label
0,4886f805,0.007859
1,0297fc1e,0.060922
2,2d681dd8,0.020292
3,cccea85e,0.091669
4,4c8a8b93,0.036538


In [55]:
#Exportamos el archivo a subir. De esta manera tiene el formato correcto. 
#Notar que así lo deja en el directorio de la notebook
clientesAEvaluar.to_csv('entrega_XGBoostBasico.csv',index=False)

#### Con la entrega anterior llegamos a 0.85443.